  
- 실행환경: macOS 12.3   
          Anaconda 4.12.0   
          Python 3.9.7   

Isolation Forest

In [22]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import RobustScaler
import joblib 


def preprocess_data(scaler_path='scaler_media.pkl', scaler=RobustScaler()):
    # 학습 데이터 읽기. 경로 설정에 주의 하세요!
    data_path = 'data/Media'
    file_list = os.listdir(data_path)

    df = []
    for i, file in enumerate(file_list):
        file_path = os.path.join(data_path, file)
        data = pd.read_csv(file_path).fillna(method='ffill').fillna(0)
        if i != 0:
            del data['Timestamp']
            data[f'{i}_tot'] = data.sum(axis=1)
        else:
            data[f'{i}_tot'] = data.drop(['Timestamp'], axis=1).sum(axis=1)
        df.append(data)
    df = pd.concat(df, axis=1)

    processed_df = df.copy()
    
    # data frame의 key를 set으로 변환하고 다시 list로 만드는 과정에서 key의 순서가 정해지지 않습니다. 
    # 본 검증 자료에서는 sort를 함으로써 동일한 결과가 나오도록 하였지만, 대회 중에는 이를 인지하지 못해 따로 통제하지 못했습니다. 
    # 이로 인해 검증 자료의 결과가 대회 기간 중 제출한 것과 완전히 동일하진 않을 수는 있으나, 결과의 유의미한 차이는 없을 것이라 
    # 판단하여 자료를 제출하니 참고 부탁드립니다.
    for key in sorted(list(set(df.keys())-set(['Timestamp']))):
        processed_df[key+'_cum'] = df[key].cumsum().fillna(method='ffill').fillna(0)
        processed_df[key+'_meddiff'] = df[key].median() - df[key]
        processed_df[key+'_mindiff'] = df[key].min() - df[key]
        processed_df[key+'_maxdiff'] = df[key].max() - df[key]
        processed_df[key+'_meandiff'] = df[key].mean() - df[key]
        processed_df[key+'_stddiff'] = df[key].std() - processed_df[key+'_meandiff']
        for i in range(1, 7):
            processed_df[key+'_diff_'+str(i)] = df[key].diff(i).fillna(method='bfill').fillna(0)
            processed_df[key+'_diff_back_'+str(i)] = df[key].diff(-i).fillna(method='ffill').fillna(0)
        for i in range(1, 3):
            processed_df[key+'_diffdays_'+str(i)] = df[key].diff(144*i).fillna(method='bfill').fillna(0)
            processed_df[key+'_diff_backdays_'+str(i)] = df[key].diff(-144*i).fillna(method='ffill').fillna(0)
    print(np.sum(processed_df.isna().to_numpy().any()))
    processed_data = scaler.fit_transform(processed_df.drop(['Timestamp'], axis=1))
    joblib.dump(scaler, scaler_path)

    # TODO: 예시코드 실행을 위한 Train_set/Test_set 분할입니다. 반드시 이 형태로 학습/테스트할 필요는 없습니다.
    end_of_year = df.index[df['Timestamp'] == '20171231_2355-0000'].tolist()[0]
    test_set = processed_data[end_of_year+1:]  # 2018 1.1 - 12.31 분리
    return processed_data, test_set

tot_data_set, test_set = preprocess_data()
joblib.dump(tot_data_set, 'data/Media/'+'tot_data_set_851.pkl')
joblib.dump(test_set, 'data/Media/'+'test_set_851.pkl')

/var/folders/5f/1pffrx8s50n84qpr5nj8k9tr0000gn/T/ipykernel_43523/437199251.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  processed_df[key+'_diff_'+str(i)] = df[key].diff(i).fillna(method='bfill').fillna(0)
/var/folders/5f/1pffrx8s50n84qpr5nj8k9tr0000gn/T/ipykernel_43523/437199251.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  processed_df[key+'_diff_back_'+str(i)] = df[key].diff(-i).fillna(method='ffill').fillna(0)
/var/folders/5f/1pffrx8s50n84qpr5nj8k9tr0000gn/T/ipykernel_43523/437199251.py:42: PerformanceWarning

0


['data/Media/test_set_851.pkl']

모델 학습 및 추론(약 30분 소요)

In [19]:
from sklearn.ensemble import IsolationForest
import joblib
import pandas as pd
import numpy as np

contamination = 0.0015
estimators = 500
model = IsolationForest(n_estimators=estimators,random_state=415, contamination=contamination)
model.fit(tot_data_set)
ans = model.predict(test_set)

ans[ans==1] = 0
ans[ans==-1] = 1
print(f'anomalies: {np.sum(ans==1)}')
answer = pd.DataFrame(ans, columns=['Prediction'])
print(f'예측 결과. \n{answer}\n')  
answer.to_csv(f'Media_answer_isolation_tree{estimators}_{contamination}_feat851.csv', index=False)  # 제출용 정답지 저장
joblib.dump(model, 'models/media_isolation_forest.pkl')

anomalies: 291
예측 결과. 
        Prediction
0                1
1                0
2                0
3                0
4                0
...            ...
105115           1
105116           1
105117           1
105118           1
105119           1

[105120 rows x 1 columns]



['models/media_isolation_forest.pkl']

predict

In [2]:
import joblib
import pandas as pd
import numpy as np

contamination = 0.0015
estimators = 500
model = joblib.load('models/media_isolation_forest.pkl')
test_set = joblib.load('data/Media/'+'test_set_851.pkl')
ans = model.predict(test_set)

ans[ans==1] = 0
ans[ans==-1] = 1
print(f'anomalies: {np.sum(ans==1)}')
answer = pd.DataFrame(ans, columns=['Prediction'])
print(f'예측 결과. \n{answer}\n')  
answer.to_csv(f'Media_answer_isolation_tree{estimators}_{contamination}_feat851.csv', index=False)  # 제출용 정답지 저장